In [1]:
import pickle
import numpy as np
import os
import keras

Using TensorFlow backend.


In [3]:
model_path = "C:/Users/sa6pr7/Desktop/Implementierung/Transfer_Learning/cpu_model.hdf5"
model = keras.models.load_model(model_path)
model.summary()

Model: "IITNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1100, 1)      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1100, 1)      0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1100, 1)      0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1100, 1)      0                                            
_____________________________________________________________________________________________

C:\Users\sa6pr7\Anaconda3\envs\jupyter_env\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
for layer in model.layers:
    print layer.tr

#### One Sample

In [7]:
p = "D:/physionet_challenge/processed/sa6pr7/training/iitnet_0/tr03-0029/distribution_matrix_preprocessed.pckl"

In [8]:
dist = pickle.load( open( p, "rb" ) )

In [9]:
dist

{'W': 335, 'N1': 134, 'N2': 304, 'N3': 21, 'R': 1}

#### Test array

In [3]:
array = np.full(4, 5)
array

array([5, 5, 5, 5])

In [5]:
range(1,1)

range(0, 1)

#### Complete directory of samples

In [121]:
#p = "D:/physionet_challenge/processed/sa6pr7/training/iitnet_0/"
#p = "D:/physionet_challenge/processed/sa6pr7/test/iitnet_0/"
#p = "D:/sleep-edf-v1/sleep-cassette/processed/training/iitnet_0/"
p = "D:/sleep-edf-v1/sleep-cassette/processed/test/iitnet_0/"

samples = [ f.path for f in os.scandir(p) if f.is_dir() ]
print("found", str(len(samples)), "folders in directory.")

found 9 folders in directory.


In [122]:
samples = [f + "/distribution_matrix_preprocessed.pckl" for f in samples]

In [123]:
dists = np.full((5), 0)    

In [81]:
missing_samples = 0
for sample in samples:
    dist = pickle.load( open( sample, "rb" ) )
    dist_l = np.asarray(list(dist.values()))
    nuller = 0
    for label in dist_l:
        if label == 0:
            nuller += 1
    if nuller == 2:
        missing_samples += 1
        
print(missing_samples)

5


In [124]:
for sample in samples[:9]:
    dist = pickle.load( open( sample, "rb" ) )
    dist_l = np.asarray(list(dist.values()))
    dists = dists + dist_l

In [125]:
dist

{'W': 331, 'N1': 337, 'N2': 327, 'N3': 57, 'R': 297}

In [126]:
dist_l

array([331, 337, 327,  57, 297])

In [127]:
dists

array([1151, 4379, 6432, 2716, 2741])

In [128]:
labels = dist.keys()

In [129]:
all_dists = dict(zip(labels, dists))
all_dists

{'W': 1151, 'N1': 4379, 'N2': 6432, 'N3': 2716, 'R': 2741}

In [130]:
sum_phases = sum(all_dists.values())
sum_phases

17419

In [131]:
percentages = np.array([x/sum_phases for x in all_dists.values()])
percentages

array([0.06607727, 0.25139216, 0.36925197, 0.15592169, 0.15735691])

#### Array Tests

In [26]:
a = np.array([[0,0,0,1,0], [0,1,0,1,0]])

In [27]:
a[0]

array([0, 0, 0, 1, 0])

In [29]:
a[:,1]

array([0, 1])

In [31]:
a = np.array([0,0,0,1])

In [32]:
b = a.tolist()

In [34]:
b.insert(3,0)

In [35]:
b

[0, 0, 0, 0, 1]

In [36]:
c = np.asarray(b)
c

array([0, 0, 0, 0, 1])

In [28]:
import tensorflow
import keras

In [29]:
from keras.layers import Input, LSTM, Dense, Conv1D
from keras.models import Model

In [39]:
model_input_a = Input(shape=(3000,1))
model_input_b = Input(shape=(3000,1))

In [40]:
model_inputs = []
model_inputs.append(model_input_a)
model_inputs.append(model_input_b)

In [41]:
layers_a = {}
layers_a['a1'] = Conv1D(8,3)
layers_a['a2'] = keras.layers.BatchNormalization()

In [42]:
layers_b = {}
layers_b['b1'] = Conv1D(8,3)
layers_b['b2'] = keras.layers.BatchNormalization()
layers_b['b3'] = keras.layers.Activation('relu')
layers_b['b4'] = Conv1D(8,3)
layers_b['b5'] = keras.layers.BatchNormalization()
layers_b['b6'] = keras.layers.Activation('relu')
layers_b['b7'] = Conv1D(16,3)
layers_b['b8'] = keras.layers.BatchNormalization()

In [45]:
layers_c = {}
layers_c['c2'] = keras.layers.Activation('relu')

In [ ]:
x1 = 0
x1_short = 0
x1_add = 0
for i, layer in enumerate(layers_a.values()):
    if i == 0:
        x1_short = layer(model_input_a)
    else:
        x_short = layer(x_short)

In [ ]:
for i, layer in enumerate(layers_b.values()):
    if i == 0:
        x1 = layer(model_input_a)
    else:
        x1 = layer(x1)

In [ ]:
for i, layer in enumerate(layers_c.values()):
    if i == 0:
        x1_add = layer(x1, x1_short)
    else:
        x1_add = layer(x1_add)

In [46]:
output_layers = []

for model_input in model_inputs:
    x1 = 0
    x1_short = 0
    x1_add = 0
    for i, layer in enumerate(layers_a.values()):
        if i == 0:
            x1_short = layer(model_input_a)
        else:
            x1_short = layer(x1_short)
    for i, layer in enumerate(layers_b.values()):
        if i == 0:
            x1 = layer(model_input_a)
        else:
            x1 = layer(x1)
    x1_add = keras.layers.add([x1, x1_short])
    for i, layer in enumerate(layers_c.values()):
        x1_add = layer(x1_add)
    output_layers.append(x1_add)

ValueError: Operands could not be broadcast together with shapes (2994, 16) (2998, 8)

In [22]:
x = 0
for i, layer in enumerate(layers):
    if i == 0:
        x = layer(model_input_a)
    else:
        x = layer(x)

W0519 10:50:30.976945 10764 deprecation_wrapper.py:119] From C:\Users\sa6pr7\Anaconda3\envs\mlbase\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.



In [23]:
xb = 0
for i, layer in enumerate(layers):
    if i == 0:
        xb = layer(model_input_b)
    else:
        xb = layer(xb)

In [7]:
branch_a = shared_bn(model_input_a)
branch_b = shared_bn(model_input_b)

W0519 10:38:53.518033 10764 deprecation_wrapper.py:119] From C:\Users\sa6pr7\Anaconda3\envs\mlbase\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [24]:
merged_vector = keras.layers.concatenate(output_layers, axis=-1)

In [25]:
predictions = Dense(1, activation='sigmoid')(merged_vector)

In [26]:
model = Model(inputs=[model_input_a, model_input_b], outputs=predictions)

In [27]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 3000, 1)      0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 3000, 1)      0                                            
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 2998, 8)      32          input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 2998, 8)      32          conv1d_5[0][0]                   
          